In [1]:
cd /content/drive/My Drive/Phony News Classifier

/content/drive/My Drive/Phony News Classifier


In [0]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
import sklearn.metrics as m
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
dataset_true=pd.read_csv('True.csv')
dataset_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [7]:
dataset_true.isnull().values.any()

False

In [8]:
dataset_fake=pd.read_csv('Fake.csv')
dataset_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [9]:
dataset_fake.isnull().values.any()

False

In [10]:
title_true=dataset_true.iloc[:,[0]]['title']
title_true

0        As U.S. budget fight looms, Republicans flip t...
1        U.S. military to accept transgender recruits o...
2        Senior U.S. Republican senator: 'Let Mr. Muell...
3        FBI Russia probe helped by Australian diplomat...
4        Trump wants Postal Service to charge 'much mor...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorities
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: title, Length: 21417, dtype: object

In [11]:
text_true=dataset_true.iloc[:,[1]]['text']
text_true

0        WASHINGTON (Reuters) - The head of a conservat...
1        WASHINGTON (Reuters) - Transgender people will...
2        WASHINGTON (Reuters) - The special counsel inv...
3        WASHINGTON (Reuters) - Trump campaign adviser ...
4        SEATTLE/WASHINGTON (Reuters) - President Donal...
                               ...                        
21412    BRUSSELS (Reuters) - NATO allies on Tuesday we...
21413    LONDON (Reuters) - LexisNexis, a provider of l...
21414    MINSK (Reuters) - In the shadow of disused Sov...
21415    MOSCOW (Reuters) - Vatican Secretary of State ...
21416    JAKARTA (Reuters) - Indonesia will buy 11 Sukh...
Name: text, Length: 21417, dtype: object

In [12]:
sent_true=title_true
len(sent_true[1])

64

In [13]:
title_fake=dataset_fake.iloc[:,[0]]['title']
title_fake

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
23476    McPain: John McCain Furious That Iran Treated ...
23477    JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478    Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479    How to Blow $700 Million: Al Jazeera America F...
23480    10 U.S. Navy Sailors Held by Iranian Military ...
Name: title, Length: 23481, dtype: object

In [14]:
text_fake=dataset_fake.iloc[:,[1]]['text']
text_fake

0        Donald Trump just couldn t wish all Americans ...
1        House Intelligence Committee Chairman Devin Nu...
2        On Friday, it was revealed that former Milwauk...
3        On Christmas day, Donald Trump announced that ...
4        Pope Francis used his annual Christmas Day mes...
                               ...                        
23476    21st Century Wire says As 21WIRE reported earl...
23477    21st Century Wire says It s a familiar theme. ...
23478    Patrick Henningsen  21st Century WireRemember ...
23479    21st Century Wire says Al Jazeera America will...
23480    21st Century Wire says As 21WIRE predicted in ...
Name: text, Length: 23481, dtype: object

In [15]:
sent_fake=title_fake
len(sent_fake[1])

69

In [16]:
len(text_fake[1])

1898

In [17]:
len(title_fake[1])

69

In [0]:
sent=sent_fake.append(sent_true,ignore_index=True)

In [19]:
sent.iloc[[-1]]

44897    Indonesia to buy $1.14 billion worth of Russia...
Name: title, dtype: object

In [0]:
label_fake=['fake']*len(sent_fake)
label_true=['true']*len(sent_true)


In [0]:
label=label_fake+label_true

In [22]:
print(label)

['fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le=LabelEncoder()
label=le.fit_transform(label)

In [25]:
label

array([0, 0, 0, ..., 1, 1, 1])

In [26]:
le.classes_

array(['fake', 'true'], dtype='<U4')

In [0]:
import re

In [28]:
len(set(stopwords.words('english')))

179

In [0]:
lemma=WordNetLemmatizer()

In [30]:
sent

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
44893    'Fully committed' NATO backs new U.S. approach...
44894    LexisNexis withdrew two products from Chinese ...
44895    Minsk cultural hub becomes haven from authorities
44896    Vatican upbeat on possibility of Pope Francis ...
44897    Indonesia to buy $1.14 billion worth of Russia...
Name: title, Length: 44898, dtype: object

In [31]:
sentences=[]
j=1
for sen in sent:
  senti=re.sub('[^A-Za-z]',' ',sen)
  senti=senti.lower()
  words=word_tokenize(senti)
  word=[lemma.lemmatize(i) for i in words if i not in stopwords.words('english')]
  senti=' '.join(word)
  sentences.append(senti)
  print("sentence ",j," executed")
  j=j+1
  


Streaming output truncated to the last 5000 lines.
sentence  39900  executed
sentence  39901  executed
sentence  39902  executed
sentence  39903  executed
sentence  39904  executed
sentence  39905  executed
sentence  39906  executed
sentence  39907  executed
sentence  39908  executed
sentence  39909  executed
sentence  39910  executed
sentence  39911  executed
sentence  39912  executed
sentence  39913  executed
sentence  39914  executed
sentence  39915  executed
sentence  39916  executed
sentence  39917  executed
sentence  39918  executed
sentence  39919  executed
sentence  39920  executed
sentence  39921  executed
sentence  39922  executed
sentence  39923  executed
sentence  39924  executed
sentence  39925  executed
sentence  39926  executed
sentence  39927  executed
sentence  39928  executed
sentence  39929  executed
sentence  39930  executed
sentence  39931  executed
sentence  39932  executed
sentence  39933  executed
sentence  39934  executed
sentence  39935  executed
sentence  399

In [32]:
sentences

['donald trump sends embarrassing new year eve message disturbing',
 'drunk bragging trump staffer started russian collusion investigation',
 'sheriff david clarke becomes internet joke threatening poke people eye',
 'trump obsessed even obama name coded website image',
 'pope francis called donald trump christmas speech',
 'racist alabama cop brutalize black boy handcuff graphic image',
 'fresh golf course trump lash fbi deputy director james comey',
 'trump said insanely racist stuff inside oval office witness back',
 'former cia director slam trump un bullying openly suggests acting like dictator tweet',
 'watch brand new pro trump ad feature much kissing make sick',
 'papa john founder retires figure racism bad business',
 'watch paul ryan told u care struggling family living blue state',
 'bad news trump mitch mcconnell say repealing obamacare',
 'watch lindsey graham trash medium portraying trump kooky forgets word',
 'heiress disney empire know gop scammed u shred tax bill',
 't

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf=TfidfVectorizer(max_features=10000)

In [0]:
features=tfidf.fit_transform(sentences)

In [0]:
features=features.toarray()

In [38]:
features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
len(tfidf.get_feature_names())

10000

In [0]:
feature_train,feature_test,label_train,label_test=train_test_split(features,label,test_size=0.2,random_state=7)

#Naive Bayies

In [41]:
model=MultinomialNB()
model.fit(feature_train,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
label_pred=model.predict(feature_test)

In [43]:
label_pred

array([1, 0, 0, ..., 0, 0, 0])

In [44]:
label_test

array([1, 0, 0, ..., 0, 0, 0])

In [45]:
m.accuracy_score(label_test,label_pred)

0.9316258351893095

In [46]:
print(m.classification_report(label_test,label_pred))

              precision    recall  f1-score   support

           0       0.93      0.94      0.94      4727
           1       0.93      0.92      0.93      4253

    accuracy                           0.93      8980
   macro avg       0.93      0.93      0.93      8980
weighted avg       0.93      0.93      0.93      8980



In [47]:
print(m.confusion_matrix(label_test,label_pred))

[[4454  273]
 [ 341 3912]]


In [0]:
import pickle

In [0]:
pickle.dump(model,open("model_lti.pkl",mode='wb+'))

In [50]:
c=model.coef_
c

array([[-10.92641442,  -8.9762324 ,  -9.20038195, ...,  -8.52755527,
        -10.02321426,  -8.08342573]])

In [51]:
model.get_params

<bound method BaseEstimator.get_params of MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)>

In [52]:
tfidf.get_feature_names()

['aarp',
 'abadi',
 'abandon',
 'abandoned',
 'abandoning',
 'abbas',
 'abbott',
 'abc',
 'abdicate',
 'abdication',
 'abducted',
 'abduction',
 'abdullah',
 'abe',
 'abedin',
 'abide',
 'abiding',
 'ability',
 'able',
 'aboard',
 'abort',
 'aborted',
 'abortion',
 'abraham',
 'abroad',
 'abrupt',
 'abruptly',
 'absence',
 'absentia',
 'absolute',
 'absolutely',
 'absurd',
 'absurdity',
 'absurdly',
 'abu',
 'abuse',
 'abused',
 'abuser',
 'abusing',
 'abusive',
 'aca',
 'academic',
 'academy',
 'accelerate',
 'accelerated',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accessing',
 'accession',
 'accident',
 'accidental',
 'accidentally',
 'accommodate',
 'accomplice',
 'accomplished',
 'accomplishment',
 'accord',
 'according',
 'account',
 'accountability',
 'accountable',
 'accounting',
 'accurate',
 'accusation',
 'accuse',
 'accused',
 'accuser',
 'accuses',
 'accusing',
 'ace',
 'achieve',
 'achievement',
 'acid',
 'ackno

In [53]:
sorted(zip(model.coef_[0],tfidf.get_feature_names()),reverse=True)

[(-4.612708806484749, 'trump'),
 (-4.728316241264397, 'say'),
 (-5.287744709146994, 'house'),
 (-5.5949928058091665, 'russia'),
 (-5.61941285547436, 'republican'),
 (-5.642542584616228, 'north'),
 (-5.652752748563706, 'korea'),
 (-5.770223894240647, 'china'),
 (-5.774274839545963, 'senate'),
 (-5.83322039799368, 'state'),
 (-5.833732982544804, 'white'),
 (-5.850275629400406, 'new'),
 (-5.860419639004221, 'court'),
 (-5.901575649480684, 'bill'),
 (-5.920171584760394, 'tax'),
 (-5.998210378169937, 'vote'),
 (-6.0047555156545, 'talk'),
 (-6.007582203556399, 'election'),
 (-6.0274932032360296, 'deal'),
 (-6.036386317965189, 'leader'),
 (-6.0648013999392365, 'eu'),
 (-6.0659083613972955, 'clinton'),
 (-6.0720372173773685, 'call'),
 (-6.080871139749686, 'official'),
 (-6.12421099719802, 'may'),
 (-6.129475376443726, 'iran'),
 (-6.165285370086547, 'obama'),
 (-6.172367796374212, 'factbox'),
 (-6.184181390815265, 'minister'),
 (-6.199769765461768, 'plan'),
 (-6.2182348706294865, 'probe'),
 (-6

In [0]:
a

[]